# Debug Drill: The Memorizing Tree

**Scenario:**
A colleague built a decision tree for churn prediction. They're confused by the results.

"The model gets 99% accuracy on training data!" they say. "But in production it's only 65%. What's going on?"

**Your Task:**
1. Run the model and diagnose the problem
2. Identify the overfitting symptoms
3. Fix the tree configuration
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
# Load data
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

customers = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(customers)} customers")

# Prepare features
if 'tenure_days' not in customers.columns:
    customers['tenure_days'] = (pd.to_datetime('2024-01-01') - pd.to_datetime(customers['signup_date'])).dt.days
if 'avg_order_value' not in customers.columns:
    customers['avg_order_value'] = customers['total_spend'] / customers['orders_total'].replace(0, 1)

feature_cols = ['tenure_days', 'orders_total', 'total_spend', 'support_tickets_total', 'avg_order_value']
available_features = [c for c in feature_cols if c in customers.columns]

X = customers[available_features].fillna(0)
y = customers['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# ===== COLLEAGUE'S CODE (CONTAINS BUG) =====

# "I want the most accurate model possible!"
tree_overfit = DecisionTreeClassifier(
    max_depth=20,         # <-- BUG: Way too deep!
    min_samples_leaf=1,   # <-- BUG: Allows single-sample leaves
    min_samples_split=2,  # <-- BUG: Splits even with 2 samples
    random_state=42
)

tree_overfit.fit(X_train, y_train)

train_acc = accuracy_score(y_train, tree_overfit.predict(X_train))
test_acc = accuracy_score(y_test, tree_overfit.predict(X_test))

print("=== Colleague's Tree ===")
print(f"  Max Depth Setting: 20")
print(f"  Actual Depth: {tree_overfit.get_depth()}")
print(f"  Number of Leaves: {tree_overfit.get_n_leaves()}")
print(f"\n  Train Accuracy: {train_acc:.1%}")
print(f"  Test Accuracy:  {test_acc:.1%}")
print(f"  Gap: {train_acc - test_acc:.1%} ← HUGE OVERFIT!")

---

## Your Investigation

The 30%+ gap between train and test accuracy is a classic overfitting symptom.

### Step 1: Understand what went wrong

In [ ]:
# How many leaves does the tree have?
print(f"Number of training samples: {len(X_train)}")
print(f"Number of tree leaves: {tree_overfit.get_n_leaves()}")
print(f"\n🤔 Ratio: {tree_overfit.get_n_leaves() / len(X_train):.1%} of training samples have their own leaf!")

if tree_overfit.get_n_leaves() > len(X_train) * 0.1:
    print("\n❌ This tree has MEMORIZED the training data.")
    print("   Each leaf represents a tiny group of similar samples.")
    print("   It learned noise, not patterns.")

In [ ]:
# Visualize the problem: accuracy vs depth
depths = range(1, 21)
results = []

for depth in depths:
    tree_temp = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree_temp.fit(X_train, y_train)
    
    results.append({
        'depth': depth,
        'train_acc': accuracy_score(y_train, tree_temp.predict(X_train)),
        'test_acc': accuracy_score(y_test, tree_temp.predict(X_test)),
        'n_leaves': tree_temp.get_n_leaves()
    })

results_df = pd.DataFrame(results)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(results_df['depth'], results_df['train_acc'], 'b-o', label='Train', linewidth=2)
ax1.plot(results_df['depth'], results_df['test_acc'], 'r-o', label='Test', linewidth=2)
ax1.axvline(x=tree_overfit.get_depth(), color='purple', linestyle='--', alpha=0.5, label='Colleague\'s depth')
ax1.fill_between(results_df['depth'], results_df['train_acc'], results_df['test_acc'], 
                 alpha=0.2, color='red', label='Overfit gap')
ax1.set_xlabel('Max Depth')
ax1.set_ylabel('Accuracy')
ax1.set_title('Train vs Test Accuracy by Depth')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(results_df['depth'], results_df['n_leaves'], 'g-s', linewidth=2)
ax2.set_xlabel('Max Depth')
ax2.set_ylabel('Number of Leaves')
ax2.set_title('Model Complexity (Leaves) by Depth')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Key observation: Test accuracy PEAKS then DROPS as depth increases!")
print("   The sweet spot is around depth 3-5, not 20.")

In [ ]:
# Find the optimal depth
optimal_idx = results_df['test_acc'].idxmax()
optimal_depth = results_df.loc[optimal_idx, 'depth']

print(f"Optimal depth (by test accuracy): {optimal_depth}")
print(f"Test accuracy at optimal: {results_df.loc[optimal_idx, 'test_acc']:.1%}")
print(f"Leaves at optimal: {results_df.loc[optimal_idx, 'n_leaves']}")

### Step 2: Fix the tree

In [ ]:
# TODO: Create a tree with proper constraints

# Uncomment and complete:

# tree_fixed = DecisionTreeClassifier(
#     max_depth=???,           # Use optimal depth from above
#     min_samples_leaf=10,     # Require at least 10 samples per leaf
#     min_samples_split=20,    # Require at least 20 samples to split
#     random_state=42
# )
# 
# tree_fixed.fit(X_train, y_train)
# 
# train_acc_fixed = accuracy_score(y_train, tree_fixed.predict(X_train))
# test_acc_fixed = accuracy_score(y_test, tree_fixed.predict(X_test))
# 
# print("=== Fixed Tree ===")
# print(f"  Max Depth: {tree_fixed.get_depth()}")
# print(f"  Number of Leaves: {tree_fixed.get_n_leaves()}")
# print(f"\n  Train Accuracy: {train_acc_fixed:.1%}")
# print(f"  Test Accuracy:  {test_acc_fixed:.1%}")
# print(f"  Gap: {train_acc_fixed - test_acc_fixed:.1%}")

In [ ]:
# TODO: Compare the trees

# Uncomment:

# print("\n=== Comparison ===")
# print(f"                    Overfit Tree    Fixed Tree")
# print(f"  Depth:            {tree_overfit.get_depth():>8}        {tree_fixed.get_depth():>8}")
# print(f"  Leaves:           {tree_overfit.get_n_leaves():>8}        {tree_fixed.get_n_leaves():>8}")
# print(f"  Train Acc:        {train_acc:>8.1%}        {train_acc_fixed:>8.1%}")
# print(f"  Test Acc:         {test_acc:>8.1%}        {test_acc_fixed:>8.1%}")
# print(f"  Gap:              {train_acc - test_acc:>8.1%}        {train_acc_fixed - test_acc_fixed:>8.1%}")
# print(f"\n  Test improvement: +{test_acc_fixed - test_acc:.1%}")

In [ ]:
# ============================================
# SELF-CHECK: Did you fix the overfitting?
# ============================================

# Uncomment:

# assert tree_fixed.get_depth() < tree_overfit.get_depth(), "Fixed tree should be shallower!"
# assert tree_fixed.get_n_leaves() < tree_overfit.get_n_leaves() / 2, "Fixed tree should have far fewer leaves!"
# assert (train_acc_fixed - test_acc_fixed) < 0.15, "Train-test gap should be < 15%"
# assert test_acc_fixed >= test_acc - 0.02, "Test accuracy should not drop significantly"
# 
# print("✓ Overfitting fixed!")
# print(f"✓ Reduced from {tree_overfit.get_n_leaves()} leaves to {tree_fixed.get_n_leaves()} leaves")
# print(f"✓ Train-test gap reduced from {train_acc - test_acc:.1%} to {train_acc_fixed - test_acc_fixed:.1%}")

### Step 3: Visualize the fixed tree

In [ ]:
# TODO: Visualize the fixed tree (should be readable!)

# Uncomment:

# plt.figure(figsize=(16, 10))
# plot_tree(
#     tree_fixed,
#     feature_names=available_features,
#     class_names=['Retained', 'Churned'],
#     filled=True,
#     rounded=True,
#     fontsize=10
# )
# plt.title('Fixed Decision Tree (Interpretable!)')
# plt.tight_layout()
# plt.show()
# 
# print("\n✓ This tree is simple enough to explain to stakeholders!")

### Step 4: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Memorizing Tree

### What happened:
- (Your answer: What symptoms indicated overfitting?)

### Root cause:
- (Your answer: Which hyperparameters were wrong and why?)

### How to prevent:
- (Your answer: What checks should we do before deploying a tree model?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Deeper ≠ Better.** Very deep trees memorize training noise instead of learning patterns.

2. **The symptom:** Train accuracy >> Test accuracy (large gap).

3. **The fixes:**
   - `max_depth`: Start with 3-5, tune with validation
   - `min_samples_leaf`: Require 5-20 samples per leaf
   - `min_samples_split`: Require 10-50 samples to split

4. **Trade off:** Lower train accuracy for better generalization.

---

## Overfitting Prevention Cheatsheet

| Symptom | Likely Cause | Fix |
|---------|-------------|-----|
| Train >> Test accuracy | Tree too deep | Reduce `max_depth` |
| Too many leaves | No leaf size constraint | Increase `min_samples_leaf` |
| Fragile predictions | Overfitting to noise | Increase `min_samples_split` |
| 100% train accuracy | Perfect memorization | All of the above |